# Optimization of a State-to-State Transfer in a Two-Level-System

This first example illustrates the basic use of the `Krotov.jl` by solving a
simple canonical optimization problem: the transfer of population in a two
level system.

In [1]:
const σ̂_z = ComplexF64[1 0; 0 -1]
const σ̂_x = ComplexF64[0 1; 1  0]



"""Two-level-system Hamiltonian."""
function hamiltonian(Ω=1.0, E0=0.2)

    Ĥ₀ = -0.5 * Ω * σ̂_z
    Ĥ₁ = σ̂_x

    ϵ(t) = 0

    return [Ĥ₀, [Ĥ₁, ϵ]]

end

H = hamiltonian()

2-element Vector{Array}:
 ComplexF64[-0.5 - 0.0im -0.0 - 0.0im; -0.0 - 0.0im 0.5 - 0.0im]
 Any[ComplexF64[0.0 + 0.0im 1.0 + 0.0im; 1.0 + 0.0im 0.0 + 0.0im], Main.##258.var"#ϵ#1"()]

The control field here switches on from zero at $t=0$ to it's maximum amplitude
0.2 within the time period 0.3 (the switch-on shape is half a [Blackman pulse](https://en.wikipedia.org/wiki/Window_function#Blackman_window)).
It switches off again in the time period 0.3 before the
final time $T=5$). We use a time grid with 500 time steps between 0 and $T$:

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*